## Load Data & Dependencies

In [5]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from ogb.nodeproppred import NodePropPredDataset
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from xgboost import XGBClassifier
import pandas as pd
import warnings

warnings.filterwarnings("ignore", message=".*weights_only=False.*")

## Load Dataset
dataset = NodePropPredDataset(name="ogbn-arxiv", root="../../data/ogb")
graph, labels = dataset[0]
X = pd.DataFrame(data=graph['node_feat'])
y = pd.DataFrame(data=labels.flatten(), columns=['label'])

## Split Data Into Train, Validation, & Test Sets

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

## Standard Baseline With Extreme Gradient Boosting

In [18]:
from sklearn.metrics import f1_score

xgb_classifier = XGBClassifier(
    objective='multi:softmax',
    eval_metric='mlogloss',
    n_estimators=200,
    learning_rate=0.1,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=1,
    n_jobs=-1,
)

xgb_classifier.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True
)


# Make predictions on validation set
val_preds = xgb_classifier.predict(X_val)

# Compute metrics
val_accuracy = accuracy_score(y_val, val_preds)
val_macro_f1 = f1_score(y_val, val_preds, average='macro')

print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation Macro-F1: {val_macro_f1:.4f}")


predictions = xgb_classifier.predict(X_test)

[0]	validation_0-mlogloss:3.27903
[1]	validation_0-mlogloss:3.07096
[2]	validation_0-mlogloss:2.91909
[3]	validation_0-mlogloss:2.79719
[4]	validation_0-mlogloss:2.69581
[5]	validation_0-mlogloss:2.60853
[6]	validation_0-mlogloss:2.53292
[7]	validation_0-mlogloss:2.46711
[8]	validation_0-mlogloss:2.40809
[9]	validation_0-mlogloss:2.35497
[10]	validation_0-mlogloss:2.30740
[11]	validation_0-mlogloss:2.26340
[12]	validation_0-mlogloss:2.22329
[13]	validation_0-mlogloss:2.18681
[14]	validation_0-mlogloss:2.15355
[15]	validation_0-mlogloss:2.12228
[16]	validation_0-mlogloss:2.09387
[17]	validation_0-mlogloss:2.06709
[18]	validation_0-mlogloss:2.04160
[19]	validation_0-mlogloss:2.01790
[20]	validation_0-mlogloss:1.99606
[21]	validation_0-mlogloss:1.97545
[22]	validation_0-mlogloss:1.95647
[23]	validation_0-mlogloss:1.93851
[24]	validation_0-mlogloss:1.92168
[25]	validation_0-mlogloss:1.90570
[26]	validation_0-mlogloss:1.89068
[27]	validation_0-mlogloss:1.87687
[28]	validation_0-mlogloss:1.8

In [16]:
# Overall metrics
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions, output_dict=True, zero_division=0)

# Macro averages
macro_f1 = report['macro avg']['f1-score']
macro_recall = report['macro avg']['recall']
macro_precision = report['macro avg']['precision']

# Weighted averages
weighted_f1 = report['weighted avg']['f1-score']
weighted_recall = report['weighted avg']['recall']
weighted_precision = report['weighted avg']['precision']

# Print neatly
print(f"Accuracy: {accuracy:.4f}")
print(f"Macro - F1: {macro_f1:.4f}, Recall: {macro_recall:.4f}, Precision: {macro_precision:.4f}")
print(f"Weighted - F1: {weighted_f1:.4f}, Recall: {weighted_recall:.4f}, Precision: {weighted_precision:.4f}")

Accuracy: 0.5470
Macro - F1: 0.3309, Recall: 0.2918, Precision: 0.4737
Weighted - F1: 0.5237, Recall: 0.5470, Precision: 0.5396


## Small Deep‑Learning Text Classifier

In [ ]:
# X_train: embeddings (num_samples, embedding_dim)
# y_train: integer class labels
num_classes = len(set(y_train))  # or your predefined number of classes

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),               # prevents overfitting
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',  # if y_train are integers
    metrics=['accuracy']
)

model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32
)

predictions = model.evaluate(X_test, y_test)

In [ ]:
# Overall metrics
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions, output_dict=True, zero_division=0)

# Macro averages
macro_f1 = report['macro avg']['f1-score']
macro_recall = report['macro avg']['recall']
macro_precision = report['macro avg']['precision']

# Weighted averages
weighted_f1 = report['weighted avg']['f1-score']
weighted_recall = report['weighted avg']['recall']
weighted_precision = report['weighted avg']['precision']

# Print neatly
print(f"Accuracy: {accuracy:.4f}")
print(f"Macro - F1: {macro_f1:.4f}, Recall: {macro_recall:.4f}, Precision: {macro_precision:.4f}")
print(f"Weighted - F1: {weighted_f1:.4f}, Recall: {weighted_recall:.4f}, Precision: {weighted_precision:.4f}")